In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import sys
sys.path.append('../')
sys.path.append('.')

from config import Config
from data_utils import video_to_frames
from data_utils import metadata_loader
from data_utils import dataset_builder

In [66]:
# config = Config(root_path='./data/1000-videos', img_width=84, img_height=84, use_subfolders=True)
config = Config()

# Get metadata
ml = metadata_loader.MetadataLoader(config)
metadata = ml.load_metadata()

# Build datasets
db = dataset_builder.DatasetBuilder(config)
train_dataset = db.make_video_dataset(metadata['train'])
valid_dataset = db.make_video_dataset(metadata['valid'])
test_dataset = db.make_video_dataset(metadata['test'])

# Resize Images

In [89]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
    #image = tf.cast(image, tf.float32)
    image.set_shape((None, config.img_height, config.img_height, 3))
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    label.set_shape([config.n_classes])
    return image, label

train = train_dataset.map(format_example)
validation = valid_dataset.map(format_example)
test = test_dataset.map(format_example)

## Set batchsize and shuffle

In [91]:
BATCH_SIZE = 20
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [92]:
for image_batch, label_batch in train_batches.take(1):
    pass
print(image_batch.shape, label_batch.shape)


(1, 85, 160, 160, 3) (1, 174)


In [93]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable=False

In [100]:
# layers
def get_model(base_model):
    
    # Layers
    global_pooling = tf.keras.layers.GlobalAveragePooling2D()
    feature_extractor = tf.keras.Sequential([base_model,global_pooling])

    features = tf.keras.layers.TimeDistributed(feature_extractor, input_shape=(None,160, 160,3))


    lstm = tf.keras.layers.LSTM(60, input_shape=(None,1280))
    lstm.build((None,None,1280))

    dense1 = tf.keras.layers.Dense(512, kernel_initializer="he_normal")

    batch_normalization = tf.keras.layers.BatchNormalization()

    classifier_dense = tf.keras.layers.Dense(174)


    # model
    full_model = tf.keras.Sequential([
        features, 
        lstm,
        dense1,
        batch_normalization,
        classifier_dense])
    
    #compile model
    base_learning_rate = 0.0001
    full_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return full_model



model = get_model(base_model)
model.summary()
print(model(image_batch).shape)


Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, None, 1280)        2257984   
_________________________________________________________________
lstm_29 (LSTM)               (None, 60)                321840    
_________________________________________________________________
dense_26 (Dense)             (None, 512)               31232     
_________________________________________________________________
batch_normalization_13 (Batc (None, 512)               2048      
_________________________________________________________________
dense_27 (Dense)             (None, 174)               89262     
Total params: 2,702,366
Trainable params: 443,358
Non-trainable params: 2,259,008
_________________________________________________________________
(1, 174)


In [98]:
full_model.fit(train_batches, epochs=14)

In [103]:
full_model.evaluate(validation_batches)

# IGNORE THOSE MUSTAFA

In [52]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)

    return loss_function(y_true=y, y_pred=y_)


def grad(model, inputs, targets):
    with tf.GradientTape(persistent=True) as tape:
        predictions = full_model(inputs)
        loss = loss_function(predictions, targets)
        
    grads = tape.gradient(loss,full_model.trainable_variable)
    return loss, tape.gradient(loss, model.trainable_variables)

In [53]:
l = loss(full_model, image_batch, label_batch, training=False)
print("Loss test: {}".format(l))

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [55]:
label_batch.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:

INPUT_SHAPE = (None,IMG_SIZE, IMG_SIZE, 3)
x = tf.keras.Input(INPUT_SHAPE)
features = tf.keras.layers.TimeDistributed(feature_extractor, input_shape=(160, 160,3))(x)

y1 = tf.keras.layers.LSTM(60, input_shape=(None,1280))(features)

y2 = tf.keras.layers.Dense(512, kernel_initializer="he_normal")(y1)
y3 = tf.keras.layers.BatchNormalization()(y2)

y = tf.keras.layers.Dense(174)(y3)

model = tf.keras.Model(x,y)
model.summary()